In [2]:
import pyAgrum as gum
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import itertools

from sklearn.model_selection import train_test_split, RepeatedKFold, GridSearchCV
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_regression
pd.set_option('display.max_rows', 200)

In [3]:
df = pd.read_csv("newOutputs/classifiers.csv", index_col=0)
df

,items,nodes,timesteps,arcs_ratio,values,algorithm,scorefunction,fscore,structure_id,PC1,...,mean_constraint,mean_efficiency,s_metric/arcs,mean_similarity,mean_nr_similarities/nodes,mean_degree,nodestates,arcs,items/nodestates,items/arcs
0,100,4,1,0.999999,2,HC,BDEU,0.333333,0,0.079587,...,0.583333,0.750000,3.000000,0.25000,0.25000,0.75,16,3,6.250000,33.333333
1,100,4,1,0.999999,2,HC,AIC,0.000000,0,0.066578,...,0.583333,0.750000,3.000000,0.25000,0.25000,0.75,16,3,6.250000,33.333333
2,100,4,1,0.999999,2,HC,BIC,0.000000,0,0.079688,...,0.583333,0.750000,3.000000,0.25000,0.25000,0.75,16,3,6.250000,33.333333
3,100,4,1,0.999999,2,HC,K2,0.000000,0,0.074820,...,0.583333,0.750000,3.000000,0.25000,0.25000,0.75,16,3,6.250000,33.333333
4,100,4,1,0.999999,2,HC,L2L,0.000000,0,0.072970,...,0.583333,0.750000,3.000000,0.25000,0.25000,0.75,16,3,6.250000,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33295,1000,8,1,1.750000,4,MMHC,BDEU,0.000000,85,0.017435,...,0.462386,0.664444,10.666667,0.18941,0.34375,1.50,65536,14,0.015259,71.428571
33296,1000,8,1,1.750000,4,MMHC,BIC,0.000000,85,0.019309,...,0.462386,0.664444,10.666667,0.18941,0.34375,1.50,65536,14,0.015259,71.428571
33297,1000,8,1,1.750000,4,MMHC,K2,0.000000,85,0.016330,...,0.462386,0.664444,10.666667,0.18941,0.34375,1.50,65536,14,0.015259,71.428571
33298,1000,8,1,1.750000,4,3OFF2,constraint,0.600000,85,0.016871,...,0.462386,0.664444,10.666667,0.18941,0.34375,1.50,65536,14,0.015259,71.428571


In [8]:
def normalize(col):
    # TODO: GROTER DAN WAT????
    if max(col) > 1:
        return col/max(col)
    return col

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']
df2 = df.copy()
df2[columns] = df2[columns].apply(normalize)
df2

df3 = pd.get_dummies(df2, columns=["algorithm","scorefunction"], dtype=int)
df3

,items,nodes,timesteps,arcs_ratio,values,fscore,structure_id,PC1,PC2,PC3,...,algorithm_HC,algorithm_MIIC,algorithm_MMHC,algorithm_TABU,scorefunction_AIC,scorefunction_BDEU,scorefunction_BIC,scorefunction_K2,scorefunction_L2L,scorefunction_constraint
0,0.1,0.5,0.333333,0.499999,0.5,0.333333,0,0.079587,0.015593,0.009940,...,1,0,0,0,0,1,0,0,0,0
1,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.066578,0.021655,0.004711,...,1,0,0,0,1,0,0,0,0,0
2,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.079688,0.017840,0.007377,...,1,0,0,0,0,0,1,0,0,0
3,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.074820,0.014163,0.005103,...,1,0,0,0,0,0,0,1,0,0
4,0.1,0.5,0.333333,0.499999,0.5,0.000000,0,0.072970,0.020888,0.007101,...,1,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33295,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.017435,0.010863,0.007120,...,0,0,1,0,0,1,0,0,0,0
33296,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.019309,0.011259,0.007205,...,0,0,1,0,0,0,1,0,0,0
33297,1.0,1.0,0.333333,0.875000,1.0,0.000000,85,0.016330,0.012039,0.006773,...,0,0,1,0,0,0,0,1,0,0
33298,1.0,1.0,0.333333,0.875000,1.0,0.600000,85,0.016871,0.013103,0.007061,...,0,0,0,0,0,0,0,0,0,1


# Linear Regression

In [9]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = LinearRegression()

regr.fit(X_train_poly, y_train)

all_score = regr.score(X_test_poly, y_test)
all_score

KeyError: "['islands'] not in index"

In [39]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

i = 1
score = 0
while score + 0.01 < all_score:
    X_train_fs, X_test_fs, fs = select_features(X_train_poly, y_train, X_test_poly, i)

    regr = LinearRegression()

    regr.fit(X_train_fs, y_train)

    score = regr.score(X_test_fs, y_test)
    print(i, score)
    i+=1

1 0.015738156802062364
2 0.07450504618778919
3 0.08033287599690364
4 0.08184055217340735
5 0.08192001638401991
6 0.09079139448381857
7 0.09113846146068683
8 0.10664039823935267
9 0.11298866140362296
10 0.1129862733657343
11 0.1297334894482669
12 0.13031252368580382
13 0.1302691219254274
14 0.13029919555942204
15 0.13533755521294533
16 0.13814044511796741
17 0.13744773048499082
18 0.16077530285920827
19 0.14347203270699704
20 0.20648503971813348


# Logistic Regression

In [16]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.5,1.0], labels=['0','1'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7082980198782649

In [17]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

for i in range(1,len(X_train.columns)):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, i)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = poly.fit_transform(X_train_fs), poly.fit_transform(X_test_fs)

    regr = LogisticRegression()

    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

0.648894367824948
0.648894367824948


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6959704137452808


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7041759765775484


...

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7067185453424764


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.7059480699591648
0.7092225903382386


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [18]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.33,0.66,1.0], labels=['0','1','2'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5615224593574235

In [19]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

for i in range(1,len(X_train.columns)):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, i)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = poly.fit_transform(X_train_fs), poly.fit_transform(X_test_fs)

    regr = LogisticRegression()

    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

0.5276215424917174


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5052777563756838


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.52846906541336


...

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5545111333692888


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.5619076970490793
0.5608290315124432


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [20]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.25,0.5,0.75,1.0], labels=['0','1','2','3'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.48405115956545186

In [21]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

for i in range(1,len(X_train.columns)):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, i)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = poly.fit_transform(X_train_fs), poly.fit_transform(X_test_fs)

    regr = LogisticRegression()

    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

0.437976731643424


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.43851606441174207


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.47372678942907775


...

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.48316511287464364


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.484860158717929
0.4882887741736652


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [22]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.2,0.4,0.6,0.8,1.0], labels=['0','1','2','3','4'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.41074042684336237

In [23]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

for i in range(1,len(X_train.columns)):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, i)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = poly.fit_transform(X_train_fs), poly.fit_transform(X_test_fs)

    regr = LogisticRegression()

    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.3576161491640342


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.3447492102627321


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.4088142383850836


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.40900685723091146
0.40631019338932123


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [24]:
dfbin = df2.copy()
dfbin["fscore"] = pd.cut(dfbin["fscore"],bins=[-0.01,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0], labels=['0','1','2','3','4','5','6','7','8','9'])

columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(dfbin[columns], dfbin["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

X_test_poly

regr = LogisticRegression(max_iter=100)

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.281377609985361

In [25]:
# feature selection
def select_features(X_train, y_train, X_test, k):
    # configure to select a subset of features
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    # learn relationship from training data
    fs.fit(X_train, y_train)
    # transform train input data
    X_train_fs = fs.transform(X_train)
    # transform test input data
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

for i in range(1,len(X_train.columns)):
    X_train_fs, X_test_fs, fs = select_features(X_train, y_train, X_test, i)

    poly = PolynomialFeatures(degree=2)
    X_train_poly, X_test_poly = poly.fit_transform(X_train_fs), poly.fit_transform(X_test_fs)

    regr = LogisticRegression()

    regr.fit(X_train_poly, y_train)

    print(regr.score(X_test_poly, y_test))

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.24793897834964174


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2299869019184837


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2783342322212805


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.2835349410586332
0.2808768009862085


C:\Users\jacco\anaconda3\envs\thesis\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


# GradientBoosting

In [12]:
columns = ['items', 'nodes', 'timesteps', 'arcs_ratio', 'islands', 'values', 'nodestates', 'arcs', 'items/nodestates', 'items/arcs']

# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(df2[columns], df2["fscore"], test_size = 0.2)

poly = PolynomialFeatures(degree=2)
X_train_poly, X_test_poly = poly.fit_transform(X_train), poly.fit_transform(X_test)

regr = GradientBoostingRegressor()

regr.fit(X_train_poly, y_train)

regr.score(X_test_poly, y_test)

0.2125780441819619